In [1]:
from Encoder import Encoder
from Decoder import Decoder
import numpy as np
import time

In [2]:
def bpsk_modulate(message):
    """
    BPSK Modulate a binary message array.
    Map 0 -> +1, and 1 -> -1.
    """
    return np.where(message == 0, 1, -1)

def add_gaussian_noise(signal, sigma):
    """
    Add Gaussian noise to the BPSK modulated signal.
    
    signal: BPSK modulated signal
    sigma: Standard deviation of the Gaussian noise
    
    Returns the received signal with noise added.
    """
    noise = np.random.normal(0, sigma, len(signal))
    return signal + noise

def add_bsc_noise(signal, p):
    """
    Adds Binary Symmetric Channel (BSC) noise to the binary 1D signal.
    
    Parameters:
    - signal: numpy array of binary values (0 or 1)
    - p: probability of flipping a bit (0 <= p <= 1)

    Returns:
    - noisy_signal: binary signal with noise added
    """
    # Generate a random array of the same length as the signal
    noise = np.random.rand(len(signal)) < p  # True (flip) with probability p
    
    # Flip bits where noise is True
    noisy_signal = np.bitwise_xor(signal, noise.astype(int))
    
    return noisy_signal


def bpsk_demodulate(received_signal):
    """
    BPSK Demodulate the received signal back to binary (0s and 1s).
    
    received_signal: Received signal with noise
    
    Returns the demodulated binary array.
    """
    return np.where(received_signal >= 0, 0, 1)



### Encoding a message into codeword

In [8]:
# base_matrix_path = 'base_matrices/NR_1_1_48.txt'
base_matrix_path = 'base_matrices/NR_1_2_20.txt'
expansion_factor = 20
en = Encoder(base_matrix_path,expansion_factor)
H = en.get_parity_check_matrix()
print("Parity Check Matrix : \n",H)

# generating codeword for a random message vector
msg = en.generate_random_message()
codeword = en.nrldpc_encode(msg)
print("message :\n",msg)
print("codeword :\n",codeword)

if(en.check_cword(codeword)):
    print("codeword is valid!")
else:
    print("codeword is invalid!")
    
if(np.sum(np.dot(H,codeword.T)%2)==0):
    print("codeword is valid!")
else:
    print("codeword is invalid!")
    print(np.sum(np.dot(H,codeword.T)%2))
    print(np.dot(H,codeword.T)%2)
    a = np.dot(H,codeword.T)%2
    print(a.shape)

Parity Check Matrix : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
message :
 [0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0
 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0
 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 0
 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 0
 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0
 0 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 0
 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0
 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 1 0 0 0 0
 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 0 1 0
 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1
 1 1 1 0 0 0 0 0 1 1

### Modulating the codeword (BPSK)

In [9]:
# Apply BPSK modulation
bpsk_signal = bpsk_modulate(codeword)
print("BPSK Modulated Signal:\n", bpsk_signal)


BPSK Modulated Signal:
 [ 1  1 -1 ...  1 -1  1]


### Adding Noise (AWGN) to the vector

In [10]:
# sigma = 0.9
# # Add Gaussian noise to the signal
# received_signal = add_gaussian_noise(bpsk_signal, sigma)
# print("Received Signal with Noise:\n", received_signal)

p = 0.3
# Add VSC noise to the signal
received_signal = add_bsc_noise(codeword, p)
print("Received Signal with Noise:\n", received_signal)


Received Signal with Noise:
 [0 1 1 ... 1 0 0]


### Demodulating the recieved vector

In [9]:
# Demodulate the received signal back to binary
demodulated_message = bpsk_demodulate(received_signal)
# print("Demodulated Binary Message:\n", demodulated_message)


In [10]:
print((demodulated_message+codeword)%2)
print("Number of errors : ",np.sum((demodulated_message+codeword)%2),"/",codeword.shape[0])

[1 0 1 ... 1 0 0]
Number of errors :  184 / 1360


### Decoding the recieved vector

In [11]:
channel_model = "bsc"
channel_parameters = [p]
num_iter = 5000
dec = Decoder(H,channel_model,channel_parameters,num_iter)

In [12]:
t1 = time.time()
c_hat = dec.decode(received_signal,100)
t2 = time.time()
print("Time taken :",t2-t1)

Iteration  0	
Iteration  100	
Iteration  200	
Iteration  300	
Iteration  400	
Iteration  500	
Iteration  600	
Iteration  700	
Iteration  800	
Iteration  900	
Iteration  1000	
Iteration  1100	
Iteration  1200	
Iteration  1300	
Iteration  1400	
Iteration  1500	
Iteration  1600	
Iteration  1700	
Iteration  1800	
Iteration  1900	
Iteration  2000	
Iteration  2100	
Iteration  2200	
Iteration  2300	
Iteration  2400	
Iteration  2500	
Iteration  2600	
Iteration  2700	
Iteration  2800	
Iteration  2900	
Iteration  3000	
Iteration  3100	
Iteration  3200	
Iteration  3300	
Iteration  3400	
Iteration  3500	
Iteration  3600	
Iteration  3700	
Iteration  3800	
Iteration  3900	
Iteration  4000	
Iteration  4100	
Iteration  4200	
Iteration  4300	
Iteration  4400	
Iteration  4500	
Iteration  4600	
Iteration  4700	
Iteration  4800	
Iteration  4900	
Time taken : 58.92180633544922


In [13]:
c_hat = np.array([1 if val else 0 for val in c_hat])
print("obtained codeword : ",c_hat)
dp = np.dot(c_hat,H.T)%2
print("Dot product : \t\t",dp)
print(en.check_cword(c_hat))

obtained codeword :  [0 0 1 ... 0 0 1]
Dot product : 		 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [14]:
print("Error fraction : ",np.sum((c_hat+codeword)%2)/c_hat.shape[0])
print(len(dp)-np.sum(dp),"/",len(dp)," Check Nodes satisfied")

Error fraction :  0.3698529411764706
907 / 920  Check Nodes satisfied


In [15]:
print((c_hat+codeword)%2)

[0 0 0 ... 0 1 1]
